 [standalone-qwen3-plus-kvcache.ipynb](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch05/11_qwen3/standalone-qwen3-plus-kvcache.ipynb)

In [3]:
from importlib.metadata import version

pkgs = [
    "huggingface_hub",
    "tokenizers",
    "torch",
]
for p in pkgs:
    print(f"{p}: {version(p)}")


huggingface_hub: 0.30.1
tokenizers: 0.21.1
torch: 2.3.1


In [ ]:
USE_BASE_MODEL = False
USE_REASONING_MODEL = True
USE_INSTRUCT_MODEL = False

if USE_BASE_MODEL + USE_REASONING_MODEL + USE_INSTRUCT_MODEL != 1:
    raise AttributeError("Exactly one of USE_BASE_MODEL, USE_REASONING_MODEL, " \
        "USE_INSTRUCT_MODEL must be True.")